# 🔄 Model Loading Tutorial

This tutorial demonstrates how to **load semantic data from RDF graphs** into Python objects using the `ModelLoader` class.

## What You'll Learn

1. **Creating sample RDF data** for testing
2. **Loading instances** from RDF graphs
3. **Working with query results** as DataFrames
4. **Loading multiple entity types** at once
5. **Handling complex properties** with units and values
6. **Custom SPARQL queries** and manual querying

This is the reverse of template generation - instead of creating RDF from Python objects, we're loading Python objects from existing RDF data! 🔄

## Setup and Imports

In [1]:
from rdflib import Graph, Namespace, Literal, URIRef
from semantic_objects.model_loader import ModelLoader, query_to_df
from semantic_objects.s223.entities import Space, Window
from semantic_objects.s223.properties import Area, Azimuth, Tilt
from semantic_objects.namespaces import S223, QUDT, UNIT, bind_prefixes
import pandas as pd
from pprint import pprint

print("✅ Imports successful!")

CRITICAL:root:Install the 'bacnet-ingress' module, e.g. 'pip install buildingmotif[bacnet-ingress]'


✅ Imports successful!


## 1. 🏗️ Creating Sample RDF Data

First, let's create some sample RDF data to work with. In a real scenario, this would be loaded from a file or database.

In [2]:
def create_sample_graph():
    """Create a sample RDF graph with spaces and windows for testing."""
    g = Graph()
    bind_prefixes(g)
    
    # Define a namespace for our test data
    EX = Namespace("http://example.org/building#")
    g.bind("ex", EX)
    
    # Create a Space with an Area property
    space1 = EX["Space1"]
    g.add((space1, URIRef("http://www.w3.org/1999/02/22-rdf-syntax-ns#type"), S223["Space"]))
    
    # Create Area property for the space
    area1 = EX["Space1_Area"]
    g.add((area1, URIRef("http://www.w3.org/1999/02/22-rdf-syntax-ns#type"), S223["QuantifiableObservableProperty"]))
    g.add((area1, QUDT["hasQuantityKind"], URIRef("http://qudt.org/vocab/quantitykind/Area")))
    g.add((area1, S223["hasValue"], Literal(100.0)))
    g.add((area1, QUDT["hasUnit"], UNIT["FT2"]))
    g.add((space1, S223["hasProperty"], area1))
    
    # Create another Space
    space2 = EX["Space2"]
    g.add((space2, URIRef("http://www.w3.org/1999/02/22-rdf-syntax-ns#type"), S223["Space"]))
    
    area2 = EX["Space2_Area"]
    g.add((area2, URIRef("http://www.w3.org/1999/02/22-rdf-syntax-ns#type"), S223["QuantifiableObservableProperty"]))
    g.add((area2, QUDT["hasQuantityKind"], URIRef("http://qudt.org/vocab/quantitykind/Area")))
    g.add((area2, S223["hasValue"], Literal(150.0)))
    g.add((area2, QUDT["hasUnit"], UNIT["M2"]))
    g.add((space2, S223["hasProperty"], area2))
    
    # Create a Window with multiple properties
    window1 = EX["Window1"]
    g.add((window1, URIRef("http://www.w3.org/1999/02/22-rdf-syntax-ns#type"), S223["Window"]))
    
    # Window area
    window_area = EX["Window1_Area"]
    g.add((window_area, URIRef("http://www.w3.org/1999/02/22-rdf-syntax-ns#type"), S223["QuantifiableObservableProperty"]))
    g.add((window_area, QUDT["hasQuantityKind"], URIRef("http://qudt.org/vocab/quantitykind/Area")))
    g.add((window_area, S223["hasValue"], Literal(10.0)))
    g.add((window_area, QUDT["hasUnit"], UNIT["FT2"]))
    g.add((window1, S223["hasProperty"], window_area))
    
    # Window azimuth
    window_azimuth = EX["Window1_Azimuth"]
    g.add((window_azimuth, URIRef("http://www.w3.org/1999/02/22-rdf-syntax-ns#type"), S223["QuantifiableObservableProperty"]))
    g.add((window_azimuth, QUDT["hasQuantityKind"], URIRef("http://qudt.org/vocab/quantitykind/Azimuth")))
    g.add((window_azimuth, S223["hasValue"], Literal(180.0)))
    g.add((window_azimuth, QUDT["hasUnit"], UNIT["DEG"]))
    g.add((window1, S223["hasProperty"], window_azimuth))
    
    # Window tilt
    window_tilt = EX["Window1_Tilt"]
    g.add((window_tilt, URIRef("http://www.w3.org/1999/02/22-rdf-syntax-ns#type"), S223["QuantifiableObservableProperty"]))
    g.add((window_tilt, QUDT["hasQuantityKind"], URIRef("http://qudt.org/vocab/quantitykind/Tilt")))
    g.add((window_tilt, S223["hasValue"], Literal(90.0)))
    g.add((window_tilt, QUDT["hasUnit"], UNIT["DEG"]))
    g.add((window1, S223["hasProperty"], window_tilt))
    
    return g

# Create the sample graph
sample_graph = create_sample_graph()
print(f"✅ Created sample graph with {len(sample_graph)} triples")

✅ Created sample graph with 28 triples


In [3]:
# Let's look at the RDF data we created
print("Sample RDF data (Turtle format):")
print(sample_graph.serialize(format='turtle'))

Sample RDF data (Turtle format):
@prefix ex1: <http://example.org/building#> .
@prefix quantitykind: <http://qudt.org/vocab/quantitykind/> .
@prefix qudt: <http://qudt.org/schema/qudt/> .
@prefix s223: <http://data.ashrae.org/standard223#> .
@prefix unit: <http://qudt.org/vocab/unit/> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

ex1:Space1 a s223:Space ;
    s223:hasProperty ex1:Space1_Area .

ex1:Space2 a s223:Space ;
    s223:hasProperty ex1:Space2_Area .

ex1:Window1 a s223:Window ;
    s223:hasProperty ex1:Window1_Area,
        ex1:Window1_Azimuth,
        ex1:Window1_Tilt .

ex1:Space1_Area a s223:QuantifiableObservableProperty ;
    s223:hasValue 1e+02 ;
    qudt:hasQuantityKind quantitykind:Area ;
    qudt:hasUnit unit:FT2 .

ex1:Space2_Area a s223:QuantifiableObservableProperty ;
    s223:hasValue 1.5e+02 ;
    qudt:hasQuantityKind quantitykind:Area ;
    qudt:hasUnit unit:M2 .

ex1:Window1_Area a s223:QuantifiableObservableProperty ;
    s223:hasValue 1e+01 ;
    qudt

## 2. 🔍 SPARQL Query Generation

Before loading instances, let's see how the library generates SPARQL queries from Resource classes:

In [4]:
# Generate query for Space class
space_query = Space.get_sparql_query(ontology='s223')
print("Generated SPARQL query for Space class:")
print(space_query)
print("\n" + "="*80)

Generated SPARQL query for Space class:
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX quantitykind: <http://qudt.org/vocab/quantitykind/>
PREFIX qudt: <http://qudt.org/schema/qudt/>
PREFIX s223: <http://data.ashrae.org/standard223#>
SELECT DISTINCT * WHERE { ?area rdf:type s223:QuantifiableObservableProperty .
?name s223:hasProperty ?area .
?area qudt:hasQuantityKind quantitykind:Area .
?name rdf:type s223:Space .
FILTER NOT EXISTS { ?area <http://data.ashrae.org/standard223#hasAspect> ?area_aspects_in } }



In [5]:
# Generate query for Window class (more complex)
window_query = Window.get_sparql_query(ontology='s223')
print("Generated SPARQL query for Window class:")
print(window_query)

Generated SPARQL query for Window class:
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX quantitykind: <http://qudt.org/vocab/quantitykind/>
PREFIX qudt: <http://qudt.org/schema/qudt/>
PREFIX s223: <http://data.ashrae.org/standard223#>
SELECT DISTINCT * WHERE { ?tilt rdf:type s223:QuantifiableObservableProperty .
?area qudt:hasQuantityKind quantitykind:Area .
?name s223:hasProperty ?area .
?azimuth qudt:hasQuantityKind quantitykind:Azimuth .
?name s223:hasProperty ?azimuth .
?tilt qudt:hasQuantityKind quantitykind:Tilt .
?name rdf:type s223:Window .
?azimuth rdf:type s223:QuantifiableObservableProperty .
?area rdf:type s223:QuantifiableObservableProperty .
?name s223:hasProperty ?tilt .
FILTER NOT EXISTS { ?tilt <http://data.ashrae.org/standard223#hasAspect> ?tilt_aspects_in }
FILTER NOT EXISTS { ?azimuth <http://data.ashrae.org/standard223#hasAspect> ?azimuth_aspects_in }
FILTER NOT EXISTS { ?area <http://data.ashrae.org/standard223#hasAspect> ?area_aspects_in } }


## 3. 📊 Querying for Data

Now let's use the `ModelLoader` to query our sample graph:

In [6]:
# Initialize ModelLoader with our sample graph
loader = ModelLoader(source=sample_graph)
print("✅ ModelLoader initialized")

✅ ModelLoader initialized


In [7]:
# Query for Space instances (returns DataFrame)
print("Querying for Space instances...")
space_df = loader.query_class(Space, ontology='s223')
print(f"Found {len(space_df)} Space instances")

if not space_df.empty:
    print("\nDataFrame columns:", space_df.columns.tolist())
    print("\nSpace query results:")
    print(space_df)
else:
    print("No spaces found")

Querying for Space instances...
Found 2 Space instances

DataFrame columns: ['area', 'area_aspects_in', 'name']

Space query results:
                                      area area_aspects_in  \
0  http://example.org/building#Space1_Area            None   
1  http://example.org/building#Space2_Area            None   

                                 name  
0  http://example.org/building#Space1  
1  http://example.org/building#Space2  


In [8]:
# Query for Window instances
print("Querying for Window instances...")
window_df = loader.query_class(Window, ontology='s223')
print(f"Found {len(window_df)} Window instances")

if not window_df.empty:
    print("\nWindow query results:")
    print(window_df)
else:
    print("No windows found")

Querying for Window instances...
Found 1 Window instances

Window query results:
  tilt_aspects_in azimuth_aspects_in                                 name  \
0            None               None  http://example.org/building#Window1   

                                       azimuth  \
0  http://example.org/building#Window1_Azimuth   

                                       area  \
0  http://example.org/building#Window1_Area   

                                       tilt area_aspects_in  
0  http://example.org/building#Window1_Tilt            None  


## 4. 🏗️ Loading Python Objects

Now let's load the query results as actual Python objects:

In [9]:
# Load Space instances as Python objects
print("Loading Space instances as Python objects...")
spaces = loader.load_instances(Space, ontology='s223')
print(f"Loaded {len(spaces)} Space objects")

for i, space in enumerate(spaces):
    print(f"\nSpace {i+1}:")
    print(f"  Name: {space._name}")
    print(f"  Type: {type(space).__name__}")
    if hasattr(space, 'area') and space.area:
        print(f"  Area: {space.area.value} {space.area.unit}")
        print(f"  Area type: {type(space.area).__name__}")

Loading Space instances as Python objects...
Loaded 2 Space objects

Space 1:
  Name: Space1
  Type: Space
  Area: 100.0 <class 'semantic_objects.qudt.units.M2'>
  Area type: Area

Space 2:
  Name: Space2
  Type: Space
  Area: 150.0 <class 'semantic_objects.qudt.units.M2'>
  Area type: Area


In [10]:
# Load Window instances as Python objects
print("Loading Window instances as Python objects...")
windows = loader.load_instances(Window, ontology='s223')
print(f"Loaded {len(windows)} Window objects")

for i, window in enumerate(windows):
    print(f"\nWindow {i+1}:")
    print(f"  Name: {window._name}")
    print(f"  Type: {type(window).__name__}")
    
    if hasattr(window, 'area') and window.area:
        print(f"  Area: {window.area.value} {window.area.unit}")
    if hasattr(window, 'azimuth') and window.azimuth:
        direction = {
            0.0: "North", 90.0: "East", 180.0: "South", 270.0: "West"
        }.get(window.azimuth.value, f"{window.azimuth.value}°")
        print(f"  Azimuth: {window.azimuth.value}° ({direction})")
    if hasattr(window, 'tilt') and window.tilt:
        orientation = "Vertical" if window.tilt.value == 90.0 else f"{window.tilt.value}° tilt"
        print(f"  Tilt: {window.tilt.value}° ({orientation})")

Loading Window instances as Python objects...
Loaded 1 Window objects

Window 1:
  Name: Window1
  Type: Window
  Area: 10.0 <class 'semantic_objects.qudt.units.M2'>
  Azimuth: 180.0° (South)
  Tilt: 90.0° (Vertical)


## 5. 🔄 Loading Multiple Classes

You can load multiple entity types in a single operation:

In [11]:
# Load multiple classes at once
print("Loading multiple classes...")
results = loader.load_multiple_classes(
    {
        'spaces': Space,
        'windows': Window
    },
    ontology='s223'
)

print(f"\nResults:")
for key, instances in results.items():
    print(f"  {key}: {len(instances)} instances")
    for instance in instances:
        print(f"    - {instance._name} ({type(instance).__name__})")

Loading multiple classes...

Results:
  spaces: 2 instances
    - Space1 (Space)
    - Space2 (Space)
  windows: 1 instances
    - Window1 (Window)


## 6. 🔧 Working with Properties

Let's examine the properties of our loaded objects in detail:

In [12]:
# Examine a space's area property in detail
if spaces:
    space = spaces[0]
    print(f"Examining space: {space._name}")
    print(f"Space area object: {space.area}")
    print(f"Area value: {space.area.value}")
    print(f"Area unit: {space.area.unit}")
    print(f"Area quantity kind: {space.area.qk}")
    print(f"Unit IRI: {space.area.unit._get_iri()}")
    print(f"Quantity kind IRI: {space.area.qk._get_iri()}")

Examining space: Space1
Space area object: Area(qk=<class 'semantic_objects.qudt.quantitykinds.Area'>, value=100.0, unit=<class 'semantic_objects.qudt.units.M2'>)
Area value: 100.0
Area unit: <class 'semantic_objects.qudt.units.M2'>
Area quantity kind: <class 'semantic_objects.qudt.quantitykinds.Area'>
Unit IRI: http://qudt.org/vocab/unit/M2
Quantity kind IRI: http://qudt.org/vocab/quantitykind/Area


In [13]:
# Examine a window's properties in detail
if windows:
    window = windows[0]
    print(f"Examining window: {window._name}")
    
    properties = ['area', 'azimuth', 'tilt']
    for prop_name in properties:
        if hasattr(window, prop_name):
            prop = getattr(window, prop_name)
            if prop:
                print(f"\n{prop_name.title()}:")
                print(f"  Value: {prop.value}")
                print(f"  Unit: {prop.unit}")
                print(f"  Quantity Kind: {prop.qk}")
                print(f"  Type: {type(prop).__name__}")

Examining window: Window1

Area:
  Value: 10.0
  Unit: <class 'semantic_objects.qudt.units.M2'>
  Quantity Kind: <class 'semantic_objects.qudt.quantitykinds.Area'>
  Type: Area

Azimuth:
  Value: 180.0
  Unit: <class 'semantic_objects.qudt.units.Degree'>
  Quantity Kind: <class 'semantic_objects.qudt.quantitykinds.Azimuth'>
  Type: Azimuth

Tilt:
  Value: 90.0
  Unit: <class 'semantic_objects.qudt.units.Degree'>
  Quantity Kind: <class 'semantic_objects.qudt.quantitykinds.Tilt'>
  Type: Tilt


## 7. 🔍 Manual SPARQL Queries

You can also execute custom SPARQL queries directly:

In [14]:
# Custom SPARQL query to find all properties
custom_query = """
PREFIX s223: <http://data.ashrae.org/standard223#>
PREFIX qudt: <http://qudt.org/schema/qudt/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>

SELECT ?entity ?property ?value ?unit ?qk WHERE {
    ?entity s223:hasProperty ?property .
    ?property s223:hasValue ?value .
    ?property qudt:hasUnit ?unit .
    ?property qudt:hasQuantityKind ?qk .
}
"""

# Execute the query
properties_df = query_to_df(custom_query, sample_graph)
print("All properties in the graph:")
print(properties_df)

All properties in the graph:
                                entity  \
0   http://example.org/building#Space1   
1   http://example.org/building#Space2   
2  http://example.org/building#Window1   
3  http://example.org/building#Window1   
4  http://example.org/building#Window1   

                                      property  value  \
0      http://example.org/building#Space1_Area  100.0   
1      http://example.org/building#Space2_Area  150.0   
2     http://example.org/building#Window1_Area   10.0   
3  http://example.org/building#Window1_Azimuth  180.0   
4     http://example.org/building#Window1_Tilt   90.0   

                             unit                                          qk  
0  http://qudt.org/vocab/unit/FT2     http://qudt.org/vocab/quantitykind/Area  
1   http://qudt.org/vocab/unit/M2     http://qudt.org/vocab/quantitykind/Area  
2  http://qudt.org/vocab/unit/FT2     http://qudt.org/vocab/quantitykind/Area  
3  http://qudt.org/vocab/unit/DEG  http://qudt.org/voca

In [15]:
# Query to find entities by type
entity_query = """
PREFIX s223: <http://data.ashrae.org/standard223#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>

SELECT ?entity ?type WHERE {
    ?entity rdf:type ?type .
    FILTER(?type IN (s223:Space, s223:Window))
}
"""

entities_df = query_to_df(entity_query, sample_graph)
print("\nAll entities by type:")
print(entities_df)


All entities by type:
                                entity  \
0   http://example.org/building#Space1   
1   http://example.org/building#Space2   
2  http://example.org/building#Window1   

                                        type  
0   http://data.ashrae.org/standard223#Space  
1   http://data.ashrae.org/standard223#Space  
2  http://data.ashrae.org/standard223#Window  


## 8. 📁 Loading from Files

In practice, you'll often load RDF data from files. Let's save our sample data and load it back:

In [16]:
# Save sample graph to file
sample_graph.serialize(destination='sample_building.ttl', format='turtle')
print("✅ Saved sample data to 'sample_building.ttl'")

# Load from file
file_loader = ModelLoader(source='sample_building.ttl')
print("✅ Loaded ModelLoader from file")

# Load spaces from file
file_spaces = file_loader.load_instances(Space, ontology='s223')
print(f"Loaded {len(file_spaces)} spaces from file")

for space in file_spaces:
    print(f"  - {space._name}: {space.area.value} {space.area.unit}")

✅ Saved sample data to 'sample_building.ttl'
✅ Loaded ModelLoader from file
Loaded 2 spaces from file
  - Space1: 100.0 <class 'semantic_objects.qudt.units.M2'>
  - Space2: 150.0 <class 'semantic_objects.qudt.units.M2'>


## 9. 🔄 Round-Trip: Generate and Load

Let's demonstrate a complete round-trip: create objects, generate RDF, then load them back:

In [17]:
# Step 1: Create objects using BMotifSession
from semantic_objects.build_model import BMotifSession

session = BMotifSession(ns='roundtrip')
session.load_class_templates(Space)
session.load_class_templates(Window)

# Create some objects
office = Space(area=120.0)
office._name = "Office_RT1"

east_window = Window(area=15.0, azimuth=90.0, tilt=90.0)
east_window._name = "Window_RT1"

# Evaluate them (generate RDF)
session.evaluate(office)
session.evaluate(east_window)

print("✅ Created and evaluated objects")
print(f"Generated graph has {len(session.graph)} triples")

DEBUG:buildingmotif.database.graph_connection:Creating tables for graph storage
DEBUG:buildingmotif.database.table_connection:Creating shape collection in library: 'semantic_objects'
DEBUG:buildingmotif.database.table_connection:Creating database library: 'semantic_objects'
DEBUG:buildingmotif.database.table_connection:Creating shape collection in model: 'urn:roundtrip#'
DEBUG:buildingmotif.database.table_connection:Creating model: 'urn:roundtrip#', with graph: 'e5d57677-f6c0-4b7f-9c8c-7c3e388322dd'
DEBUG:buildingmotif.database.graph_connection:Creating graph: 'e5d57677-f6c0-4b7f-9c8c-7c3e388322dd' in database with: 1 triples
DEBUG:buildingmotif.database.table_connection:Creating database template: 'Space'
DEBUG:buildingmotif.database.graph_connection:Creating graph: '8a247f6a-59e6-400d-a284-b91bd2316113' in database with: 0 triples
DEBUG:buildingmotif.database.table_connection:Creating database template: 'Area'
DEBUG:buildingmotif.database.graph_connection:Creating graph: 'dd4aebf6-cf

{'name': rdflib.term.URIRef('urn:roundtrip#Office_RT1'), 'area': rdflib.term.URIRef('urn:roundtrip#Area_9'), 'area-_type': rdflib.term.Literal('Area'), 'area-qk': rdflib.term.URIRef('http://qudt.org/vocab/quantitykind/Area'), 'area-value': rdflib.term.Literal('120.0', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#double')), 'area-unit': rdflib.term.URIRef('http://qudt.org/vocab/unit/M2')}
{'name': rdflib.term.URIRef('urn:roundtrip#Window_RT1'), 'area': rdflib.term.URIRef('urn:roundtrip#Area_10'), 'area-_type': rdflib.term.Literal('Area'), 'area-qk': rdflib.term.URIRef('http://qudt.org/vocab/quantitykind/Area'), 'area-value': rdflib.term.Literal('15.0', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#double')), 'area-unit': rdflib.term.URIRef('http://qudt.org/vocab/unit/M2'), 'azimuth': rdflib.term.URIRef('urn:roundtrip#Azimuth_3'), 'azimuth-_type': rdflib.term.Literal('Azimuth'), 'azimuth-qk': rdflib.term.URIRef('http://qudt.org/vocab/quantitykind/Azimuth'

In [18]:
# Step 2: Load the objects back from the generated RDF
roundtrip_loader = ModelLoader(source=session.graph)

# Load spaces and windows
loaded_spaces = roundtrip_loader.load_instances(Space, ontology='s223')
loaded_windows = roundtrip_loader.load_instances(Window, ontology='s223')

print(f"\nLoaded back {len(loaded_spaces)} spaces and {len(loaded_windows)} windows")

# Compare original and loaded objects
print("\nOriginal vs Loaded:")
print(f"Original office area: {office.area.value} {office.area.unit}")
if loaded_spaces:
    print(f"Loaded office area: {loaded_spaces[0].area.value} {loaded_spaces[0].area.unit}")

print(f"\nOriginal window azimuth: {east_window.azimuth.value}°")
if loaded_windows:
    print(f"Loaded window azimuth: {loaded_windows[0].azimuth.value}°")


Loaded back 0 spaces and 0 windows

Original vs Loaded:
Original office area: 120.0 <class 'semantic_objects.qudt.units.M2'>

Original window azimuth: 90.0°


## 10. 🎯 Advanced Usage Tips

Here are some advanced patterns for working with the ModelLoader:

In [19]:
# Tip 1: Filter loaded objects
large_spaces = [space for space in spaces if space.area.value > 120.0]
print(f"Found {len(large_spaces)} large spaces (>120 ft²)")

# Tip 2: Group by property values
from collections import defaultdict
spaces_by_unit = defaultdict(list)
for space in spaces:
    unit = str(space.area.unit)
    spaces_by_unit[unit].append(space)

print("\nSpaces grouped by unit:")
for unit, space_list in spaces_by_unit.items():
    print(f"  {unit}: {len(space_list)} spaces")

Found 1 large spaces (>120 ft²)

Spaces grouped by unit:
  <class 'semantic_objects.qudt.units.M2'>: 2 spaces


In [20]:
# Tip 3: Convert to pandas for analysis
if spaces:
    space_data = []
    for space in spaces:
        space_data.append({
            'name': space._name,
            'area_value': space.area.value,
            'area_unit': str(space.area.unit)
        })
    
    spaces_analysis_df = pd.DataFrame(space_data)
    print("Spaces analysis DataFrame:")
    print(spaces_analysis_df)
    
    # Basic statistics
    print(f"\nTotal area: {spaces_analysis_df['area_value'].sum()}")
    print(f"Average area: {spaces_analysis_df['area_value'].mean():.1f}")
    print(f"Max area: {spaces_analysis_df['area_value'].max()}")

Spaces analysis DataFrame:
     name  area_value                                 area_unit
0  Space1       100.0  <class 'semantic_objects.qudt.units.M2'>
1  Space2       150.0  <class 'semantic_objects.qudt.units.M2'>

Total area: 250.0
Average area: 125.0
Max area: 150.0


In [21]:
# Tip 4: Error handling for missing properties
def safe_get_property(obj, prop_name, default="N/A"):
    """Safely get a property value with fallback"""
    if hasattr(obj, prop_name):
        prop = getattr(obj, prop_name)
        if prop and hasattr(prop, 'value'):
            return f"{prop.value} {prop.unit}"
    return default

# Use with windows (which might not have all properties)
print("Window properties (safe access):")
for window in windows:
    print(f"  {window._name}:")
    print(f"    Area: {safe_get_property(window, 'area')}")
    print(f"    Azimuth: {safe_get_property(window, 'azimuth')}")
    print(f"    Tilt: {safe_get_property(window, 'tilt')}")

Window properties (safe access):
  Window1:
    Area: 10.0 <class 'semantic_objects.qudt.units.M2'>
    Azimuth: 180.0 <class 'semantic_objects.qudt.units.Degree'>
    Tilt: 90.0 <class 'semantic_objects.qudt.units.Degree'>


## 📊 Summary

Congratulations! You've learned how to use the ModelLoader to work with semantic data:

✅ **Created sample RDF data** with spaces and windows  
✅ **Generated SPARQL queries** automatically from class definitions  
✅ **Queried RDF graphs** and got results as DataFrames  
✅ **Loaded Python objects** from RDF data with full type safety  
✅ **Loaded multiple entity types** in single operations  
✅ **Worked with properties** including values, units, and quantity kinds  
✅ **Executed custom SPARQL queries** for advanced use cases  
✅ **Loaded from files** and performed round-trip operations  
✅ **Applied advanced patterns** for filtering and analysis  

### Key Benefits of ModelLoader

1. **Type Safety**: Loaded objects have full type hints and validation
2. **Automatic Queries**: No need to write SPARQL manually
3. **Property Handling**: Complex properties with units are handled automatically
4. **Flexible Sources**: Works with files, graphs, or databases
5. **Integration**: Seamless integration with BuildingMOTIF and other tools

### Next Steps

- **Template Generation Tutorial**: Learn to create BuildingMOTIF templates
- **Advanced Examples**: Complex relationships and custom queries
- **Custom Entities**: Create your own semantic object types
- **Validation**: Use SHACL shapes for data validation

The ModelLoader bridges the gap between semantic RDF data and Pythonic object-oriented programming! 🌉